# Transcription Json -> csv 전환 작업

In [ ]:
import os
import json
import csv

input_dir = "json_dir"  # JSON 파일이 들어 있는 폴더
output_csv = "train.csv"
audio_base_path = "data/"  # wav 파일이 존재하는 경로

data = []

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(input_dir, filename)

    with open(filepath, 'r', encoding='utf-8') as f:
        j = json.load(f)

    # 오디오 파일 경로
    base_audio_name = os.path.basename(j['mediaUrl'])
    audio_path_sd = os.path.join(audio_base_path, base_audio_name)

    # SN 버전도 존재할 경우 함께 처리
    audio_path_sn = audio_path_sd.replace("_SD.wav", "_SN.wav")

    # 모든 대화 내용을 하나의 문장으로 합치기
    full_text = " ".join([dialog['speakerText'].strip() for dialog in j['dialogs']])

    # SD 버전
    data.append((audio_path_sd, full_text))

    # SN 버전
    if os.path.exists(audio_path_sn):  # SN 파일이 실제 존재할 경우
        data.append((audio_path_sn, full_text))

# CSV로 저장
with open(output_csv, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["path", "text"])
    writer.writerows(data)

print(f"총 {len(data)} 개 항목 저장 완료 → {output_csv}")


# Training

In [ ]:
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import pandas as pd
import os

model_name = "openai/whisper-small"
language = "ko"
task = "transcribe"

# 경로 설정
base_dir = os.path.dirname(__file__)
data_csv_path = os.path.join(base_dir, "metadata/train.csv")
output_dir = os.path.join(base_dir, "fine-tuned-model")

# 데이터셋 로드
dataset = load_dataset("csv", data_files={"train": data_csv_path}, delimiter=",")
dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

# 모델 & 전처리기
processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# 데이터 전처리 함수
def prepare_dataset(batch):
    audio = batch["path"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=16000).input_features[0]
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset["train"].column_names)

# 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="no",
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    tokenizer=processor.feature_extractor,
)

trainer.train()

# 모델 저장
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)